In [28]:
import pandas as pd
import numpy as np

file_paths = [
    '../../doc1/Data/yfinance_data/AAPL_historical_data.csv',
    '../../doc1/Data/yfinance_data/AMZN_historical_data.csv',
    '../../doc1/Data/yfinance_data/GOOG_historical_data.csv',
    '../../doc1/Data/yfinance_data/META_historical_data.csv',
    '../../doc1/Data/yfinance_data/MSFT_historical_data.csv',
    '../../doc1/Data/yfinance_data/NVDA_historical_data.csv',
    '../../doc1/Data/yfinance_data/TSLA_historical_data.csv',
]

stock_symbols = ['Stock_1', 'Stock_2', 'Stock_3', 'Stock_4', 'Stock_5', 'Stock_6', 'Stock_7']


In [29]:

all_data = {}


for i, file_path in enumerate(file_paths):
    df = pd.read_csv(file_path)
    
   
    df = df.drop(columns=['Dividends', 'Stock Splits'])

   
    df['Stock'] = stock_symbols[i]

    
    all_data[stock_symbols[i]] = df

In [30]:
def calculate_moving_averages(df, short_window=50, long_window=200):
    df['MA50'] = df['Close'].rolling(window=short_window).mean()
    df['MA200'] = df['Close'].rolling(window=long_window).mean()
    return df

# Apply moving averages to each dataset
for stock in all_data:
    all_data[stock] = calculate_moving_averages(all_data[stock])


In [31]:
def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df


for stock in all_data:
    all_data[stock] = calculate_rsi(all_data[stock])


In [32]:
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    short_ema = df['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = df['Close'].ewm(span=long_window, adjust=False).mean()
    df['MACD'] = short_ema - long_ema
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    return df

for stock in all_data:
    all_data[stock] = calculate_macd(all_data[stock])


In [33]:
def calculate_returns(df):
    df['Daily_Return'] = df['Close'].pct_change()
    return df


for stock in all_data:
    all_data[stock] = calculate_returns(all_data[stock])


In [35]:
def moving_averages_insights(df):
    # Get the latest values of the moving averages
    latest_ma50 = df['MA50'].iloc[-1]
    latest_ma200 = df['MA200'].iloc[-1]
    
    # Compare the latest values
    if latest_ma50 > latest_ma200:
        trend = "Uptrend"
    else:
        trend = "Downtrend"
    
    return latest_ma50, latest_ma200, trend

# Apply to each stock and print the results
for stock in all_data:
    ma50, ma200, trend = moving_averages_insights(all_data[stock])
    print(f"{stock}: 50-Day MA = {ma50:.2f}, 200-Day MA = {ma200:.2f}, Trend = {trend}")


Stock_1: 50-Day MA = 210.86, 200-Day MA = 188.91, Trend = Uptrend
Stock_2: 50-Day MA = 187.04, 200-Day MA = 167.81, Trend = Uptrend
Stock_3: 50-Day MA = 180.17, 200-Day MA = 154.02, Trend = Uptrend
Stock_4: 50-Day MA = 491.47, 200-Day MA = 431.02, Trend = Uptrend
Stock_5: 50-Day MA = 439.18, 200-Day MA = 402.56, Trend = Uptrend
Stock_6: 50-Day MA = 119.38, 200-Day MA = 79.57, Trend = Uptrend
Stock_7: 50-Day MA = 206.11, 200-Day MA = 204.48, Trend = Uptrend


In [36]:
def rsi_insights(df):
    # Get the latest RSI value
    latest_rsi = df['RSI'].iloc[-1]
    
    # Interpret the RSI value
    if latest_rsi > 70:
        rsi_status = "Overbought"
    elif latest_rsi < 30:
        rsi_status = "Oversold"
    else:
        rsi_status = "Neutral"
    
    return latest_rsi, rsi_status

# Apply to each stock and print the results
for stock in all_data:
    rsi, status = rsi_insights(all_data[stock])
    print(f"{stock}: RSI = {rsi:.2f}, Status = {status}")


Stock_1: RSI = 28.94, Status = Oversold
Stock_2: RSI = 22.69, Status = Oversold
Stock_3: RSI = 23.15, Status = Oversold
Stock_4: RSI = 26.09, Status = Oversold
Stock_5: RSI = 21.60, Status = Oversold
Stock_6: RSI = 21.04, Status = Oversold
Stock_7: RSI = 34.48, Status = Neutral


In [37]:
def macd_insights(df):
    # Get the latest MACD and Signal line values
    latest_macd = df['MACD'].iloc[-1]
    latest_signal = df['MACD_Signal'].iloc[-1]
    
    # Interpret the MACD and Signal line values
    if latest_macd > latest_signal:
        signal = "Bullish"
    else:
        signal = "Bearish"
    
    return latest_macd, latest_signal, signal

# Apply to each stock and print the results
for stock in all_data:
    macd, signal, direction = macd_insights(all_data[stock])
    print(f"{stock}: MACD = {macd:.2f}, Signal = {signal:.2f}, Direction = {direction}")


Stock_1: MACD = 2.01, Signal = 4.30, Direction = Bearish
Stock_2: MACD = -2.28, Signal = -0.99, Direction = Bearish
Stock_3: MACD = -2.71, Signal = -0.65, Direction = Bearish
Stock_4: MACD = -10.36, Signal = -6.30, Direction = Bearish
Stock_5: MACD = -5.18, Signal = -1.27, Direction = Bearish
Stock_6: MACD = -2.95, Signal = -0.39, Direction = Bearish
Stock_7: MACD = 5.21, Signal = 10.49, Direction = Bearish


In [38]:
def returns_insights(df):
    # Calculate the average daily return and standard deviation
    avg_return = df['Daily_Return'].mean()
    std_dev = df['Daily_Return'].std()
    
    return avg_return, std_dev

# Apply to each stock and print the results
for stock in all_data:
    avg_return, std_dev = returns_insights(all_data[stock])
    print(f"{stock}: Average Daily Return = {avg_return:.4f}, Standard Deviation = {std_dev:.4f}")


Stock_1: Average Daily Return = 0.0011, Standard Deviation = 0.0279
Stock_2: Average Daily Return = 0.0017, Standard Deviation = 0.0354
Stock_3: Average Daily Return = 0.0010, Standard Deviation = 0.0193
Stock_4: Average Daily Return = 0.0013, Standard Deviation = 0.0243
Stock_5: Average Daily Return = 0.0011, Standard Deviation = 0.0211
Stock_6: Average Daily Return = 0.0019, Standard Deviation = 0.0379
Stock_7: Average Daily Return = 0.0020, Standard Deviation = 0.0359
